In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import json
import pandas as pd
from argparse import Namespace

# open steam search
# open Network Web Inspector and Debugger 
# scorll down, you will findout '?query' line
# Look at it's Headers, you can find the bellow link 

url = 'https://store.steampowered.com/search/results/?query&start=0&count=50&dynamic_data=&sort_by=_ASC&supportedlang=english&snr=1_7_7_230_7&infinite=1'

# the request will return a json object, you can see it if open in browser
r = requests.get(url)
data = r.json()

# this object has 4 key
# there are two key that we consider
# 'results_html' including each parser_element of game
# 'total_count' is total of game
print(data.keys())
print(data['total_count'])

dict_keys(['success', 'results_html', 'total_count', 'start'])
129344


In [2]:
soup = BeautifulSoup(data['results_html'],'html.parser')
games = soup.find_all('a')
print(type(games[0]) , type(games[0]) == bs4.element.Tag) 

<class 'bs4.element.Tag'> True


In [4]:
# get information from Data['results_html']

game_list = []

# for each game, we get id, name, platform,release_date, url, price, disprice, review
for game in games[:5]:
    idx = int(game.get('data-ds-appid'))
    name = game.find('span',{'class':'title'}).text
    platform = ', '.join(game.find('span',{'class':"platform_img"}).get('class')[1:])
    vr_supported = 'True' if game.find('span',{'class':"VR Supported"}) else False
    try: 
        release_date = game.find('div',{'class':"search_released"}).text
    except:
        release_date = None
    url = game.get('href').split('?snr=')[0]
    price = game.find('div',{'class':"search_price"}).text.strip().split('₫')[0]
    if price == 'Free to Play':
        price = '0'
    try:
        disprice = game.find('div',{'class':"search_price"}).text.strip().split('₫')[1]
    except:
        disprice = price
    review = game.find('span',{'class':"search_review_summary"}).get('data-tooltip-html')

    return_dict = {
        'id' : idx,
        'name' : name,
        'platform' : platform,
        'vr_supported': vr_supported,
        'release_date':release_date,
        'url':url,
        'price':price,
        'disprice':disprice,
        'review':review
    }    
    with open("/mnt/d/Github/Steam_ETL/output_example/parse_task.txt",'a+') as f:
        f.write(str(return_dict)+'\n')
    print(str(return_dict))

{'id': 1326470, 'name': 'Sons Of The Forest', 'platform': 'win', 'vr_supported': False, 'release_date': 'Feb 23, 2023', 'url': 'https://store.steampowered.com/app/1326470/Sons_Of_The_Forest/', 'price': '$29.99', 'disprice': '$29.99', 'review': 'Very Positive<br>82% of the 68,497 user reviews for this game are positive.'}
{'id': 1085660, 'name': 'Destiny 2', 'platform': 'win', 'vr_supported': False, 'release_date': 'Oct 1, 2019', 'url': 'https://store.steampowered.com/app/1085660/Destiny_2/', 'price': 'Free To Play', 'disprice': 'Free To Play', 'review': 'Very Positive<br>83% of the 524,564 user reviews for this game are positive.'}
{'id': 730, 'name': 'Counter-Strike: Global Offensive', 'platform': 'win', 'vr_supported': False, 'release_date': 'Aug 21, 2012', 'url': 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/', 'price': '0', 'disprice': '0', 'review': 'Very Positive<br>88% of the 6,972,389 user reviews for this game are positive.'}
{'id': 990080, 'name': 'Ho

In [74]:
gamesdf = pd.DataFrame(game_list)
gamesdf.to_csv('game_prices.csv',index=False)
print('Saved to csv')

Saved to csv


In [2]:
import mysql.connector
mydb = mysql.connector.connect(
  host="10.210.58.45",
  user="wsl_percent",
  password="pass",
  allow_local_infile=True
)
mycursor = mydb.cursor()

In [4]:
# test case
command_ = """select distinct user from mysql.user;
"""
mycursor = mydb.cursor()
mycursor.execute(command_.replace('\n'," "))

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('wsl_percent',)
('mysql.infoschema',)
('mysql.session',)
('mysql.sys',)
('root',)
('wsl_local',)
